## Escenario pruebas 2 - Incluyendo nueva logica KM

In [1]:

import pandas as pd
import logging
import traceback
import json
from typing import Dict, List, Optional, Any
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage
from graph.perfil.state import (EstadoAnalisisPerfil, PerfilUsuario, InfoPasajeros, FiltrosInferidos, EconomiaUsuario, InfoClimaUsuario)
from utils.enums import (Transmision, NivelAventura, TipoUsoProfesional,  DimensionProblematica, EstiloConduccion, TipoMecanica, FrecuenciaUso, DistanciaTrayecto, FrecuenciaViajesLargos)

test_config = {"configurable": { "thread_id": "test_thread_notebook_1234"}}
from graph.perfil.nodes import (
    # calcular_recomendacion_economia_modo1_node, # Asumimos que existe pero no lo usaremos directamente si el modo es 2
   aplicar_filtros_pasajeros_node, aplicar_filtros_pasajeros_node,inferir_filtros_node, calcular_flags_dinamicos_node,calcular_pesos_finales_node, formatear_tabla_resumen_node, calcular_km_anuales_postprocessing_node, buscar_coches_finales_node)

try:
    from config.llm import llm, llm_explicacion_coche
except ImportError:
    print("WARN: No se pudo importar la configuración de LLM. Las explicaciones podrían no funcionar.")
    llm_explicacion_coche = None

# Configurar logging básico para ver los DEBUG de los nodos
logging.basicConfig(level=logging.INFO)
logging.getLogger().setLevel(logging.INFO)

# --- Configuración de Pandas para Mejor Visualización ---
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', 150)


Entorno local detectado. Verificando GOOGLE_APPLICATION_CREDENTIALS del archivo .env...


In [2]:

PERFIL_A_PROBAR = "PERFIL" 
print(f"--- 🚗 CONFIGURANDO PRUEBA PARA PERFIL: {PERFIL_A_PROBAR} ---")

if PERFIL_A_PROBAR == "PERFIL":
    test_info_clima = InfoClimaUsuario(
        MUNICIPIO_ZBE=False, ZONA_LLUVIAS=False, ZONA_NIEBLAS=False, ZONA_NIEVE=False,
        ZONA_CLIMA_MONTA=False, ZONA_GLP=False, ZONA_GNV=False, cp_valido_encontrado=True
    )
    test_preferencias_usuario = PerfilUsuario(
        apasionado_motor="no", #activa premium y singular
        valora_estetica="si",  #activa singular
        prefiere_diseno_exclusivo="no", #activa singular
        circula_principalmente_ciudad = 'si',
        uso_profesional="no", 
        tipo_uso_profesional=None, #TipoUsoProfesional.PASAJEROS/CARGA/MIXTO
        altura_mayor_190="si", peso_mayor_100="no",
        coche_principal_hogar="si",
        rating_fiabilidad_durabilidad=8,
        rating_seguridad=7,
        rating_comodidad=8,
        rating_impacto_ambiental=7,
        rating_costes_uso=8, 
        rating_tecnologia_conectividad=10,
        transporta_carga_voluminosa="no", 
        necesita_espacio_objetos_especiales= "no",
        arrastra_remolque="no", 
        tiene_garage="si",
        espacio_sobra_garage = "si",  
        problemas_aparcar_calle= "no", # DimensionProblematica.LARGO/ANCHO/ALTO 
        tiene_punto_carga_propio="no",
        solo_electricos="no", 
        prioriza_baja_depreciacion="no",
        frecuencia_uso=FrecuenciaUso.DIARIO,
        distancia_trayecto=DistanciaTrayecto.MENOS_10_KM , 
        realiza_viajes_largos = 'no',
        frecuencia_viajes_largos= None, #FrecuenciaViajesLargos.OCASIONALMENTE,
        transmision_preferida=Transmision.AMBOS,
        estilo_conduccion=EstiloConduccion.MIXTO,
        aventura=NivelAventura.extrema,
        
    ) ##Si suele_llevar_acompanantes= false: info_pasajeros.frecuencia a "nunca".["nunca", "ocasional", "frecuente"]
    test_info_pasajeros = InfoPasajeros(suele_llevar_acompanantes= False ,
                                frecuencia= None, 
                                frecuencia_viaje_con_acompanantes = None, #ocasional , frecuente
                                num_ninos_silla=0, num_otros_pasajeros=0) 
    
    
    
    test_economia_usuario = EconomiaUsuario(modo=2, submodo=1, pago_contado=12000.0)


# 
initial_filtros_inferidos =  FiltrosInferidos( 
     #tipo_mecanica=[TipoMecanica.GASOLINA, TipoMecanica.DIESEL, TipoMecanica.GLP , TipoMecanica.GNV , TipoMecanica.HEVD , TipoMecanica.HEVG , TipoMecanica.PHEVG , TipoMecanica.PHEVD ] # Ejemplo
)

initial_state_dict: EstadoAnalisisPerfil = {
    "messages": [AIMessage(content="...")], 
    "info_clima_usuario": test_info_clima,
    "preferencias_usuario": test_preferencias_usuario, 
    "info_pasajeros": test_info_pasajeros,
    "economia": test_economia_usuario, 
    "filtros_inferidos": initial_filtros_inferidos,
    "pesos": None, "tabla_resumen_criterios": None, "coches_recomendados": None,
    "pregunta_pendiente": None, "penalizar_puertas_bajas": False, "priorizar_ancho": False,
    # Flags que se calcularán en calcular_flags_dinamicos_node
    "flag_penalizar_low_cost_comodidad": False, #Activando flags penalización comodidad flag penalizar depor comod y flag penalizar lowcost comod
    "flag_penalizar_deportividad_comodidad": False, #Activando flags penalización comodidad flag penalizar depor comod y flag penalizar lowcost comod
    "flag_penalizar_antiguo_por_tecnologia": False, # Lógica para Flag de Penalización por Antigüedad y Tecnología > 7
    "aplicar_logica_distintivo_ambiental": False,
    "es_municipio_zbe": False,
    "codigo_postal_extraido_temporal": None,
    "tipo_mensaje_cp_llm": None,
    "_decision_cp_validation": None,
    "penalizar_bev_reev_aventura_ocasional": False,
    "penalizar_phev_aventura_ocasional": False,
    "penalizar_electrificados_aventura_extrema": False,
    "favorecer_carroceria_montana": False,
    'favorecer_carroceria_comercial': False,
    'favorecer_carroceria_pasajeros_pro': False,
    'desfavorecer_carroceria_no_aventura': False, #Aventura 'ninguna' y no es clima de montaña. Activando flag para desfavorecer PICKUP/TODOTERRENO."
    'favorecer_suv_aventura_ocasional': False,
    "favorecer_pickup_todoterreno_aventura_extrema": False,
    "aplicar_logica_objetos_especiales": False,
    "favorecer_carroceria_confort": False,
    "flag_logica_uso_ocasional": False,
    "flag_favorecer_bev_uso_definido": False,
    "flag_penalizar_phev_uso_intensivo":False,
    "flag_favorecer_electrificados_por_punto_carga": False,  
    'penalizar_awd_ninguna_aventura' : False,
    'flag_bonus_awd_clima_adverso' : False,
    'favorecer_awd_aventura_ocasional' : False,
    'favorecer_awd_aventura_extrema' : False,
    'flag_logica_reductoras_aventura': False,
    'flag_logica_diesel_ciudad' : False,
    
}
# --- 3. Ejecutar la Secuencia de Nodos de Finalización y Búsqueda ---
print("\n--- 🚀 INICIANDO SECUENCIA DE FINALIZACIÓN Y BÚSQUEDA ---")
current_state = initial_state_dict.copy()
try: 
    aplicar_filtros_pasajeros_node
    print("\n0. aplicar_filtros_pasajeros_node...")
    output_filtros = aplicar_filtros_pasajeros_node(current_state)
    current_state.update(output_filtros)
    
    print("\n0. Infiriendo filtros previos...")
    output_filtros = inferir_filtros_node(current_state)
    current_state.update(output_filtros)
    #print(f"✅ filtros aplicado: {current_state.get('km_anuales_estimados')}")
       
    print("\n1. Calculando KM anuales...")
    output_km_anuales = calcular_km_anuales_postprocessing_node(current_state)
    current_state.update(output_km_anuales)
    print(f"✅ KM Anuales Estimados: {current_state.get('km_anuales_estimados')}")

    print("\n2. Calculando Flags dinámicos...")
    output_flags = calcular_flags_dinamicos_node(current_state)
    current_state.update(output_flags)

    print("\n3. Calculando Pesos finales...")
    output_pesos = calcular_pesos_finales_node(current_state)
    current_state.update(output_pesos)

    print("\n4. Generando Tabla Resumen de Criterios...")
    # Asumimos que tienes un nodo o función para esto
    # Si no, esta parte se puede comentar
    output_tabla = formatear_tabla_resumen_node(current_state)
    current_state.update(output_tabla)
    
    print("\n5. 🚀 Ejecutando Búsqueda Final en BigQuery...")
    output_final_busqueda = buscar_coches_finales_node(current_state , test_config)
    current_state.update(output_final_busqueda)
    
    print("\n\n--- 🏁 RESULTADO FINAL ---")
    print("\n--- Mensaje Final del Agente (Combinado) ---")
    if current_state.get('messages'):
        current_state['messages'][-1].pretty_print()
    else:
        print("No se encontraron coches que cumplan los criterios.")
        # Imprimir el mensaje de sugerencia que genera el nodo
        if current_state.get('messages'):
            current_state['messages'][-1].pretty_print()
except Exception as e_test:
    print(f"\n❌ ERROR durante la prueba de la secuencia de nodos: {e_test}")
    traceback.print_exc()

INFO:root:PostProc Filtros: Restricción geográfica -> Eliminando GLP.
INFO:root:PostProc Filtros: Restricción geográfica -> Eliminando GNV.
INFO:root:--- FIN DEBUG PostProc Filtros --- Filtros de mecánica finales construidos: [<TipoMecanica.BEV: 'BEV'>, <TipoMecanica.DIESEL: 'DIESEL'>, <TipoMecanica.FCEV: 'FCEV'>, <TipoMecanica.GASOLINA: 'GASOLINA'>, <TipoMecanica.HEVD: 'HEVD'>, <TipoMecanica.HEVG: 'HEVG'>, <TipoMecanica.MHEVD: 'MHEVD'>, <TipoMecanica.MHEVG: 'MHEVG'>, <TipoMecanica.PHEVD: 'PHEVD'>, <TipoMecanica.PHEVG: 'PHEVG'>, <TipoMecanica.REEV: 'REEV'>]
INFO:root:DEBUG (CalcKM) ► Cálculo: (52 * 10 * 10) + (0 * 0) = 5200 km/año
INFO:root:DEBUG (CalcFlags) ► Aventura EXTREMA. Activando flag para favorecer PICKUP/TODOTERRENO.
INFO:root:DEBUG (CalcFlags) ► Nivel Aventura 'extrema'. Activando bonus para ALL.
INFO:root:DEBUG (CalcFlags) ► Nivel Aventura 'extrema'. Activando bonus alto para Reductoras.
INFO:root:DEBUG (CalcFlags) ► Rating Comodidad (8) >= 8. Activando flag para favorecer 

--- 🚗 CONFIGURANDO PRUEBA PARA PERFIL: PERFIL ---

--- 🚀 INICIANDO SECUENCIA DE FINALIZACIÓN Y BÚSQUEDA ---

0. aplicar_filtros_pasajeros_node...
--- Ejecutando Nodo: aplicar_filtros_pasajeros_node ---
DEBUG (Aplicar Filtros Pasajeros) ► Info recibida: freq='None', X=0, Z=0

0. Infiriendo filtros previos...
--- Ejecutando Nodo: inferir_filtros_node/construir_filtros_node ---
DEBUG (Filtros) ► Preferencias e info_clima disponibles. Construyendo filtros...
DEBUG (Filtros) ► Filtros finales construidos: tipo_mecanica=[<TipoMecanica.BEV: 'BEV'>, <TipoMecanica.DIESEL: 'DIESEL'>, <TipoMecanica.FCEV: 'FCEV'>, <TipoMecanica.GASOLINA: 'GASOLINA'>, <TipoMecanica.HEVD: 'HEVD'>, <TipoMecanica.HEVG: 'HEVG'>, <TipoMecanica.MHEVD: 'MHEVD'>, <TipoMecanica.MHEVG: 'MHEVG'>, <TipoMecanica.PHEVD: 'PHEVD'>, <TipoMecanica.PHEVG: 'PHEVG'>, <TipoMecanica.REEV: 'REEV'>] tipo_carroceria=None modo_adquisicion_recomendado=None precio_max_contado_recomendado=None cuota_max_calculada=None plazas_min=None

1. Calcul

INFO:root:✅ BigQuery query ejecutada, 10 resultados obtenidos.
INFO:root:INFO (Buscar BQ) ► 10 coches crudos se limpian NaN para ->  10 coches.
INFO:root:Log de búsqueda para conversación 'test_thread_notebook_1234' insertado en BigQuery.


INFO (BQ Logger) ► Log para 'test_thread_notebook_1234' guardado en BQ.


--- 🏁 RESULTADO FINAL ---

--- Mensaje Final del Agente (Combinado) ---
================================== Ai Message ==================================

✅ He entendido lo siguiente sobre tus preferencias:

| Preferencia              | Valor                      |
|--------------------------|----------------------------|
| Código Postal            | No proporcionado |
|Condiciones Zona  | Generales / No específicas 
| Apasionado del motor    | No 
| Estética                | Importante 
| Principal del Hogar     | Sí 
| Uso                     | Particular 
| Tipo de coche           | No necesariamente eléctrico 
| Diseño exclusivo        | No (Discreto) |
| Altura                  | Mayor a 1.90 m 
| Peso                    | Menor a 100 kg  
| Transmisión preferida   | Ambos      
| Aventura                | Extrema    
| Prioriza Baja Depreciación| No 

📊 Importancia de Características (0-10):

| Característic

In [3]:
import pandas as pd
coches_recomendados = current_state[('coches_recomendados')]

df_resultados = pd.DataFrame(coches_recomendados)
        
#  OPCIÓN 1: Imprimir el DataFrame directamente (Pandas usará las opciones de display)
# print("\n--- Vista DataFrame (Pandas Display Options) ---")
# display(df_resultados) # 'display()' suele dar mejor formato en notebooks que 'print(df)'

#OPCIÓN 2: Convertir a Markdown (como en tu nodo, pero para consola)
print("\n--- Vista Markdown ---")
columnas_deseadas = [
    # Columnas principales
    'nombre', 'ID', 'marca', 'modelo', 'ano_unidad', 'anos_vehiculo', 'tipo_carroceria' ,'reductoras',  'score_total', 'puntuacion_base', 'ajustes_experto',
    # Desglose de puntuacion_base
    'dbg_score_estetica', 'estetica', 'dbg_score_premium', 'premium' ,'dbg_score_singular', 'singular', 'dbg_score_altura_libre', 'altura_libre',
    'dbg_score_batalla', 'batalla', 'dbg_score_altura_interior','indice_altura_interior',  'dbg_score_ancho', 'ancho' ,'dbg_score_fiabilidad', 'fiabilidad', 'dbg_score_durabilidad' , 'durabilidad',
    'dbg_score_seguridad', 'seguridad' , 'dbg_score_comodidad', 'comodidad','dbg_score_tecnologia', 'tecnologia' ,'dbg_score_devaluacion', 'devaluacion',
    'dbg_score_maletero_min', 'maletero_minimo','dbg_score_maletero_max', 'maletero_maximo' ,'dbg_score_largo', 'largo','dbg_score_autonomia_max', 'autonomia_uso_maxima',
    'dbg_score_bajo_peso', 'peso' ,'dbg_score_bajo_consumo', 'indice_consumo_energia' ,'dbg_score_bajo_coste_uso', 'costes_de_uso',
    'dbg_score_bajo_coste_mantenimiento', 'costes_mantenimiento',
    'dbg_score_par_remolque', 'par' ,
    'dbg_score_remolque_cf', 'capacidad_remolque_con_freno',
    'dbg_score_remolque_sf',  'capacidad_remolque_sin_freno',
    'dbg_score_menor_superficie', 'superficie_planta',
    'dbg_score_menor_giro', 'diametro_giro' ,
    'dbg_score_menor_largo', 'largo',
    'dbg_score_menor_ancho', 'ancho',
    'dbg_score_menor_alto', 'alto',
    'dbg_score_deportividad', 'deportividad',
    'dbg_score_menor_rel_peso_pot', 'relacion_peso_potencia',
    'dbg_score_potencia', 'potencia_maxima',
    'dbg_score_par_deportivo', 'par',
    'dbg_score_autonomia_principal', 'autonomia_uso_principal',
    'dbg_score_autonomia_2nd', 'autonomia_uso_2nd_drive',
    'dbg_score_menor_t_carga', 'tiempo_carga_min',
    'dbg_score_pot_carga_ac',  'potencia_maxima_carga_ac',
    'dbg_score_pot_carga_dc',  'potencia_maxima_carga_dc',
    'dbg_score_menor_aceleracion' ,'aceleracion_0_100','dbg_pen_km_extremo' , 'dbg_pen_puertas' , 'dbg_pen_low_cost_comodidad' , 'dbg_pen_deportividad_comodidad' ,
                'dbg_pen_antiguedad' , 'dbg_ajuste_distintivo' , 'dbg_bonus_ocasion_ambiental' , 'dbg_ajuste_zbe' ,
                'dbg_pen_bev_reev_avent_ocas' , 'dbg_pen_phev_avent_ocas' , 'dbg_pen_electrif_avent_extr' ,
                'dbg_bonus_car_montana' , 'dbg_bonus_car_comercial' , 'dbg_bonus_car_pasajeros' , 'dbg_pen_car_no_aventura' ,
                'dbg_bonus_suv_avent_ocas' , 'dbg_bonus_tt_avent_extr' , 'dbg_bonus_pickup_avent_extr' , 
                'dbg_ajuste_objetos_especiales' , 'dbg_bonus_car_confort' , 'dbg_bonus_ocasion_uso_ocas' ,
                'dbg_pen_electrif_uso_ocas' , 'dbg_bonus_bev_uso_definido' , 'dbg_pen_phev_uso_intensivo' ,
                'dbg_bonus_punto_carga' , 'dbg_ajuste_awd_aventura' , 'dbg_bonus_awd_nieve' , 'dbg_bonus_awd_montana' ,
                'dbg_bonus_reductoras' , 'dbg_ajuste_diesel_ciudad' , 'dbg_ajuste_km_anuales'
]
columnas_existentes = [col for col in columnas_deseadas if col in df_resultados.columns]
if columnas_existentes:
            if 'score_total' in df_resultados.columns: # Formatear score para legibilidad
                df_resultados['score_total'] = df_resultados['score_total'].apply(lambda x: f"{x:.4f}" if isinstance(x, float) else x)
            print(df_resultados[columnas_existentes].to_markdown(index=False))
else:
            print("WARN: No se encontraron columnas esperadas para mostrar en formato tabla.")
            print("Resultados crudos (lista de dicts):", coches_recomendados)


--- Vista Markdown ---
| nombre                                             |     ID | marca      | modelo     |   ano_unidad |   anos_vehiculo | tipo_carroceria   | reductoras   |   score_total |   puntuacion_base |   ajustes_experto |   dbg_score_estetica |   estetica |   dbg_score_premium |   premium |   dbg_score_singular |   singular |   dbg_score_altura_libre |   dbg_score_batalla |   batalla |   dbg_score_altura_interior |   indice_altura_interior |   dbg_score_ancho |   ancho |   dbg_score_fiabilidad |   fiabilidad |   dbg_score_durabilidad |   durabilidad |   dbg_score_seguridad |   seguridad |   dbg_score_comodidad |   comodidad |   dbg_score_tecnologia |   tecnologia |   dbg_score_devaluacion |   devaluacion |   dbg_score_maletero_min |   maletero_minimo |   dbg_score_maletero_max |   maletero_maximo |   dbg_score_largo |   largo |   dbg_score_autonomia_max |   autonomia_uso_maxima |   dbg_score_bajo_peso |   peso |   dbg_score_bajo_consumo |   indice_consumo_energia |   db

In [ ]:
print(current_state.keys())

In [ ]:
print(current_state['codigo_postal_usuario'])
print('-----------------------')
print(current_state['info_clima_usuario'])
print('-----------------------')
print(current_state['preferencias_usuario'])
print('-----------------------')
print(current_state['filtros_inferidos'])
print('-----------------------')
print(current_state['info_pasajeros'])
print('-----------------------')
print(current_state['economia'])
print('-----------------------')
print(current_state['pesos'])
print('-----------------------')
print("info_pasajeros:",current_state['info_pasajeros'])
print('-----------------------')

In [7]:
print("flag_penalizar_deportividad_comodidad:", current_state['flag_penalizar_deportividad_comodidad'])
print('-----------------------')
print("flag_penalizar_low_cost_comodidad:", current_state['flag_penalizar_low_cost_comodidad'])
print('-----------------------')
print("penalizar_puertas_bajas:" , current_state['penalizar_puertas_bajas'])
print('-----------------------')
print("flag_penalizar_antiguo_por_tecnologia:" , current_state['flag_penalizar_antiguo_por_tecnologia'])
print('-----------------------')
print("aplicar_logica_distintivo_ambiental:" ,  current_state['aplicar_logica_distintivo_ambiental'])
print('-----------------------')
print("es municipio_zbe: " , current_state['es_municipio_zbe'])
print('-----------------------')
print("penalizar_electrificados_aventura_extrema: " , current_state['penalizar_electrificados_aventura_extrema'])
print('-----------------------')
print('favorecer_carroceria_montana:', current_state['favorecer_carroceria_montana'])
print('-----------------------')
print('favorecer_carroceria_comercial', current_state['favorecer_carroceria_comercial'])
print('-----------------------')
print('favorecer_carroceria_pasajeros_pro:', current_state['favorecer_carroceria_pasajeros_pro'])
print('-----------------------')
print('aplicar_logica_objetos_especiales:', current_state['aplicar_logica_objetos_especiales'])#OK 
print('-----------------------')
print('favorecer_carroceria_confort:', current_state['favorecer_carroceria_confort']) #OK
print('-----------------------')
print('flag_logica_uso_ocasional:' , current_state['flag_logica_uso_ocasional'])
print('-----------------------')
print('flag_favorecer_bev_uso_definido:' , current_state['flag_favorecer_bev_uso_definido'])
print('-----------------------')
print('flag_penalizar_phev_uso_intensivo:' , current_state['flag_penalizar_phev_uso_intensivo'])
print('-----------------------')
print('flag_favorecer_electrificados_por_punto_carga:' , current_state['flag_favorecer_electrificados_por_punto_carga'])
print('-----------------------')
print('km_anuales_estimados:' , current_state['km_anuales_estimados'])

flag_penalizar_deportividad_comodidad: True
-----------------------
flag_penalizar_low_cost_comodidad: True
-----------------------
penalizar_puertas_bajas: False
-----------------------
flag_penalizar_antiguo_por_tecnologia: True
-----------------------
aplicar_logica_distintivo_ambiental: True
-----------------------
es municipio_zbe:  False
-----------------------
penalizar_electrificados_aventura_extrema:  True
-----------------------
favorecer_carroceria_montana: False
-----------------------
favorecer_carroceria_comercial False
-----------------------
favorecer_carroceria_pasajeros_pro: False
-----------------------
aplicar_logica_objetos_especiales: False
-----------------------
favorecer_carroceria_confort: True
-----------------------
flag_logica_uso_ocasional: False
-----------------------
flag_favorecer_bev_uso_definido: False
-----------------------
flag_penalizar_phev_uso_intensivo: False
-----------------------
flag_favorecer_electrificados_por_punto_carga: False
--------

In [8]:

print('desfavorecer_carroceria_no_aventura:', current_state['desfavorecer_carroceria_no_aventura']) #Aventura nula y no en montaña desfavorece PICKUP/TODOTERRENO
print('-----------------------')
print('favorecer_suv_aventura_ocasional:', current_state['favorecer_suv_aventura_ocasional']) #OK
print('-----------------------')
print('favorecer_pickup_todoterreno_aventura_extrema:', current_state['favorecer_pickup_todoterreno_aventura_extrema'])
print('-----------------------')
print("penalizar_bev_reev_aventura_ocasional: " , current_state['penalizar_bev_reev_aventura_ocasional'])
print('-----------------------')
print("penalizar_phev_aventura_ocasional: " , current_state['penalizar_phev_aventura_ocasional'])
print('-----------------------')
print('penalizar_awd_ninguna_aventura:' , current_state['penalizar_awd_ninguna_aventura'])
print('-----------------------')
print('favorecer_awd_aventura_ocasional:' , current_state['favorecer_awd_aventura_ocasional'])
print('-----------------------')
print('favorecer_awd_aventura_extrema:' , current_state['favorecer_awd_aventura_extrema'])
print('-----------------------')
print('flag_bonus_awd_nieve:' , current_state['flag_bonus_awd_nieve'])
print('-----------------------')
print('flag_bonus_awd_montana:' , current_state['flag_bonus_awd_montana'])
print('-----------------------')
print('flag_logica_reductoras_aventura:' , current_state['flag_logica_reductoras_aventura'])
print('-----------------------')
print('flag_logica_diesel_ciudad:' , current_state['flag_logica_diesel_ciudad'])
print('-----------------------') 
print('flag_bonus_awd_clima_adverso:' , current_state['flag_bonus_awd_clima_adverso']) 



desfavorecer_carroceria_no_aventura: False
-----------------------
favorecer_suv_aventura_ocasional: False
-----------------------
favorecer_pickup_todoterreno_aventura_extrema: True
-----------------------
penalizar_bev_reev_aventura_ocasional:  False
-----------------------
penalizar_phev_aventura_ocasional:  False
-----------------------
penalizar_awd_ninguna_aventura: False
-----------------------
favorecer_awd_aventura_ocasional: False
-----------------------
favorecer_awd_aventura_extrema: True
-----------------------
flag_bonus_awd_nieve: False
-----------------------
flag_bonus_awd_montana: False
-----------------------
flag_logica_reductoras_aventura: True
-----------------------
flag_logica_diesel_ciudad: PENALIZAR
-----------------------
flag_bonus_awd_clima_adverso: False


In [ ]:
df_resultados[columnas_existentes].to_csv(
    "resultados_filtrados.csv",
    index=False,
    sep=";",
    float_format="%.4f",
    encoding="utf-8"
)


df_resultados.to_csv(
    "resultados_filtradosV2.csv",
    index=False,
    sep=";",
    float_format="%.4f",
    encoding="utf-8"
)
